In [1]:
# taken partially from tutorial at: https://stmorse.github.io/journal/espn-fantasy-python.html
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import requests

league1 = 380134
league2 = 1553391
leagues=[league1, league2]

weeks = 3

scores={}

for i in range(len(leagues)):
    scores[i] = {}
for week in range(1, weeks+1):
    for league in range(len(leagues)):
        r = requests.get('http://games.espn.com/ffl/api/v2/scoreboard', 
                         params={'leagueId': leagues[league], 'seasonId': 2018, 'matchupPeriodId': week})
        scores[league][week] = r.json()

df = []
# test = pd.DataFrame.from_dict(scores[1], orient='index')
# test.reset_index(level=0, inplace=True)
# display(test)
# print(scores[1])
for league in range(len(leagues)):
    for key in scores[league]:
        temp = scores[league][key]['scoreboard']['matchups']
        for match in temp:
#           create ints to determine who won
            home = 0
            away = 0
            if match["winner"]=="home":
                home=1
                away=-1
            else:
                home=-1
                away=1
                
            df.append([key, 
                       match['teams'][0]['team']['teamAbbrev'],
                       match['teams'][1]['team']['teamAbbrev'],
                       match['teams'][0]['score'],
                       match['teams'][1]['score'],
                       home,
                       away
                      ])
df = pd.DataFrame(df, columns=['Week', 'HomeAbbrev', 'AwayAbbrev', 'HomeScore', 'AwayScore', 'Home Record', 'Away Record'])
df.head()
df = (df[['Week', 'HomeAbbrev', 'HomeScore', 'Home Record']]
      .rename(columns={'HomeAbbrev': 'Abbrev', 'HomeScore': 'Score', 'Home Record':'# Wins - # Losses'})
      .append(df[['Week', 'AwayAbbrev', 'AwayScore', 'Away Record']]
             .rename(columns={'AwayAbbrev': 'Abbrev', 'AwayScore': 'Score', 'Away Record' : '# Wins - # Losses'}))
     )
df1 = df.groupby("Abbrev")["Score"].describe().drop("count", axis=1)
s2 = df.groupby("Abbrev")['# Wins - # Losses'].sum()
s1 = df.groupby("Abbrev")['Score'].sum()
finalDF = df1.join(s2).join(s1).rename(columns={'Score':"Total Score"})
#  df.sort_values(by=['col1', 'col2'])
finalDF = finalDF.sort_values(by=['# Wins - # Losses','Total Score'], ascending=False)
finalDF = finalDF.reset_index()
finalDF.index +=1
display(finalDF)

,Abbrev,mean,std,min,25%,50%,75%,max,# Wins - # Losses,Total Score
1,PETR,110.506667,69.550340,47.54,73.18,98.82,141.99,185.16,3,331.52
2,HARN,86.353333,49.760291,29.28,69.21,109.14,114.89,120.64,3,259.06
3,FITZ,85.473333,43.678632,35.18,71.26,107.34,110.62,113.90,3,256.42
4,NATH,67.166667,39.715852,24.00,49.67,75.34,88.75,102.16,3,201.50
5,LURI,89.940000,56.465653,24.74,73.49,122.24,122.54,122.84,1,269.82
6,DREW,89.840000,59.766158,20.88,71.44,122.00,124.32,126.64,1,269.52
7,LEE,83.673333,41.742239,35.80,69.28,102.76,107.61,112.46,1,251.02
8,AUTO,79.760000,38.388576,44.70,59.25,73.80,97.29,120.78,1,239.28
9,ROCK,78.673333,45.800036,26.34,62.29,98.24,104.84,111.44,1,236.02
10,DDC,71.500000,33.460377,45.82,52.58,59.34,84.34,109.34,1,214.50
